In [15]:
from data_cleaning_mimic.data_cleaning_mimic_class import MedicalDataProcessor
from data_cleaning_mimic.data_cleaning_function.create_dataframe import *
from Neo4jDatabase.Neo4jDatabase_class import Neo4jDatabase
from llm_models.llm_model_class import LLM_Models
from ablation_study.ablation_study_class import AblationEval
from stability_evaluation.stability_evaluation_class import StabilityEval
from robustness_evaluation.robustness_evaluation_class import RobustnessEval
from readability_evaluation.readability_evaluation_class import ReadabilityEval
import pandas as pd
from config import config

## Initialize Model and Data Cleaning Processor

In [2]:
# Initialize the LLM model 
llm_model = LLM_Models()
# Initialize the MedicalDataProcessor instance
processor = MedicalDataProcessor(llm_model, 20)
# Cleaning the raw EHR data
processor.orchestrator()

## Import to Neo4j Local Database

In [3]:
processor.final_data["patient"] = pd.read_csv(f'{config["data_path"]}/patients.csv')
processor.final_data["admission"] = pd.read_csv(f'{config["data_path"]}/admissions.csv')
processor.final_data["family_history"] = pd.read_csv(f'{config["data_path"]}/family_history.csv')
processor.final_data["history"] = pd.read_csv(f'{config["data_path"]}/history.csv')
processor.final_data["social_history"] = pd.read_csv(f'{config["data_path"]}/social_history.csv')
processor.final_data["symptoms"] = pd.read_csv(f'{config["data_path"]}/symptoms.csv')
processor.final_data["vitals"] = pd.read_csv(f'{config["data_path"]}/vitals.csv')
processor.final_data["allergies"] = pd.read_csv(f'{config["data_path"]}/allergies.csv')

In [4]:
database = Neo4jDatabase(
        uri=config["db_uri"],
        user=config["db_user"],
        password=config["db_password"]
    )
# Upload to Neo4j Database
database.db_creation_orchestrator(
    df_patients=processor.final_data["patient"],
    df_admission=processor.final_data["admission"],
    df_symptoms=processor.final_data["symptoms"],
    df_history=processor.final_data["history"],
    df_vitals=processor.final_data["vitals"],
    df_allergies=processor.final_data["allergies"],
    df_social_history=processor.final_data["social_history"],
    df_family_history=processor.final_data["family_history"]
)

## QA set for Evaluation

In [17]:
QAset= pd.read_csv("/Users/huiziyu/Dropbox/AIPatient_Analysis/data/clean_data/QA_set.csv")

# Begin Ablation Study

In [14]:
ablation_study = AblationEval(database, llm_model, QAset)
ablation_study.orchestrator()
ablation_study.evaluator()

## Begin Stability Evaluation

In [19]:
stability_evaluation = StabilityEval(database, llm_model, QAset)
stability_evaluation.orchestrator()
stability_evaluation.evaluator()

## Begin Robustness Evaluation

In [22]:
robustness_evaluation = RobustnessEval(database, llm_model, QAset)
robustness_evaluation.orchestrator()
robustness_evaluation.evaluator()

## Begin Readability Evaluation

In [ ]:
readability_evaluation = ReadabilityEval(database, llm_model, QAset)
readability_evaluation.orchestrator()
readability_evaluation.evaluator()